In [1]:
import copy

In [2]:
with open("./data/day03-test.txt", "r") as f:
    schematics = f.read().splitlines()

In [3]:
schematics

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

In [4]:
def read_numbers(schematics):
    # replace all digits with an increasing index
    numbers = []
    for i, line in enumerate(schematics):
        numbers.append([])
        for j, char in enumerate(line):
            if char.isdigit():
                numbers[i].append(1)
            else:
                numbers[i].append(0)

    return numbers


In [5]:
def read_symbols(schematics):
    # replace all symbols with 1
    symbols = []
    for i, line in enumerate(schematics):
        symbols.append([])
        for j, char in enumerate(line):
            if char.isdigit():
                symbols[i].append(0)
            elif char == ".":
                symbols[i].append(0)
            else:
                symbols[i].append(1)
    return symbols

In [6]:
def read_gears(schematics):
    # replace all symbols with 1
    symbols = []
    for i, line in enumerate(schematics):
        symbols.append([])
        for j, char in enumerate(line):
            if char == "*":
                symbols[i].append(1)
            else:
                symbols[i].append(0)
    return symbols

In [7]:
def check_neighborhood(field, i, j):
    # check if there are numbers adjacent
    for ii in range(i-1, i+2):
        for jj in range(j-1, j+2):
            # print(ii, jj)
            if ii >= 0 and ii < len(field) and jj >= 0 and jj < len(field[i]):
                if field[ii][jj] > 0:
                    field[ii][jj] *= -1

def check_adjacency(numbers, symbols):
    adjacent = copy.deepcopy(numbers)
    # change all numbers that are adjacent to a symbol to 2
    for i, line in enumerate(adjacent):
        for j, _num in enumerate(line):
            if symbols[i][j] > 0:
                check_neighborhood(adjacent, i, j)

    return adjacent

# for each line in numbers, replace contiguous 1s with an ascending index
def index_contiguous(numbers):
    # replace all contiguous 1s with an increasing index
    counter = 1
    for i, line in enumerate(numbers):
        j_from = -1
        for j, num in enumerate(line):
            if num == 1:
                if j_from < 0:
                    j_from = j
            elif num == 0 or j == len(line) - 1:
                if j_from == -1:
                    continue
                for jj in range(j_from, j):
                    numbers[i][jj] = counter
                counter += 1
                j_from = -1

def propagate(numbers, i, j):
    for jj in range(j, len(numbers[i])):
        if numbers[i][jj] == 0:
            break
        numbers[i][jj] = numbers[i][j]
    for jj in range(j, -1, -1):
        if numbers[i][jj] == 0:
            break
        numbers[i][jj] = numbers[i][j]

def sign_contiguous(numbers):
    counter = 1
    for i, line in enumerate(numbers):
        for j, num in enumerate(line):
            if num < 0:
                propagate(numbers, i, j)

# def check_gear_neighborhood(field, i, j):
#     # check if there are exactly two numbers adjacent
#     numbers_seen = 1
#     for ii in range(i-1, i+2):
#         for jj in range(j-1, j+2):
#             # print(ii, jj)
#             if ii >= 0 and ii < len(field) and jj >= 0 and jj < len(field[i]):
#                 if field[ii][jj] == 1:
#                     field[ii][jj] = -1 * numbers_seen
#                     convert_row(field, ii, jj, -1 * numbers_seen)
#                     numbers_seen += 1

In [8]:
numbers = read_numbers(schematics)
index_contiguous(numbers)
symbols = read_symbols(schematics)
gears = read_gears(schematics)

adjacent_symbols = check_adjacency(numbers, symbols)
sign_contiguous(adjacent_symbols)

adjacent_gears = check_adjacency(numbers, gears)
sign_contiguous(adjacent_gears)

In [9]:
def pick_engine_parts(schematics, adjacent):
    engine_parts = []
    for i, line in enumerate(schematics):
        number = ""
        for j, char in enumerate(line):
            # print(i,j)
            if adjacent[i][j] < 0:
                number += char
            elif adjacent[i][j] == 0 and number != "":
                engine_parts.append(int(number))
                number = ""
            else:
                number = ""
        if number != "":
            engine_parts.append(int(number))
    return engine_parts

In [16]:
def pick_eligible_gears(schmeatics, gears, adjacent):
    all_gears = []
    eligible_gears = []
    for i, line in enumerate(gears):
        for j, char in enumerate(line):
            if gears[i][j] == 1:
                all_gears.append((i, j))

    for i, j in all_gears:
        diff_numbers = set()
        for ii in range(i-1, i+2):
            for jj in range(j-1, j+2):
                if ii >= 0 and ii < len(adjacent) and jj >= 0 and jj < len(adjacent[i]):
                    if adjacent[ii][jj] < 0:
                        diff_numbers.add(adjacent[ii][jj])
        if len(diff_numbers) == 2:
            tmp = []
            for num in diff_numbers:
                print(num, type(num))
                number = ""
                # print(adjacent)
                for i in range(len(adjacent)):
                    for j in range(len(adjacent[i])):
                        # print(i, j, adjacent[i][j])
                        if adjacent[i][j] == num:
                            print(i, j, adjacent[i][j])
                            number += schematics[i][j]

                tmp.append(int(number))
            eligible_gears.append(tmp[0] * tmp[1])
    return eligible_gears

In [17]:
adjacent_gears

[[-1, -1, -1, 0, 0, 2, 2, 2, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, -3, -3, 0, 0, 4, 4, 4, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [-5, -5, -5, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 6, 6, 0],
 [0, 0, 7, 7, 7, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, -8, -8, -8, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 9, 9, 9, 0, -10, -10, -10, 0, 0]]

In [18]:
pick_eligible_gears(schematics, gears, adjacent_gears)

-3 <class 'int'>
2 2 -3
2 3 -3
-3 35


IndexError: list index out of range

In [42]:
engine_parts = pick_engine_parts(schematics, adjacent_symbols)
sum(engine_parts)

4361

In [46]:
def solve(schematics):
    numbers = read_numbers(schematics)
    index_contiguous(numbers)
    symbols = read_symbols(schematics)
    gears = read_gears(schematics)

    adjacent_symbols = check_adjacency(numbers, symbols)
    sign_contiguous(adjacent_symbols)
    engine_parts = pick_engine_parts(schematics, adjacent_symbols)
    return sum(engine_parts)

In [47]:
with open("./data/day03.txt", "r") as f:
    schematics = f.read().splitlines()

In [48]:
solve(schematics)

529618